## load prerequisites

In [1]:
import csv
import random
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or 

## Preprocess data : names

In [2]:
data_name = set()
max_len = 0
with open('../data/woman_name_dataset.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            tmp_name = row[1].split()[0]
            data_name.add(row[1].split()[0])
            if len(tmp_name) > max_len:
                max_len = len(tmp_name)
data_name = list(data_name)
print('name total : {}'.format(len(data_name)))
print('maximum name length : {}'.format(max_len))

name total : 1219
maximum name length : 11


## Preprocess data : Characters

In [3]:
chars = set()
for name in data_name:
    for char in name:
        chars.add(char)
chars = list(np.sort(list(chars)))
print('{} alphabets : '.format(len(chars)), chars)

26 alphabets :  ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Define function to convert name to onehot

In [4]:
def name_to_onehot(names, chars, max_len):
    onehot = np.zeros((len(names), max_len, len(chars)+1))
    for idx_1, name in enumerate(names):
        for idx_2 in range(max_len):
            if idx_2 < len(name):
                idx_3 = chars.index(name[idx_2])
                onehot[idx_1, idx_2, idx_3] = 1
            else:
                onehot[idx_1, idx_2, -1] = 1
    return onehot

onehot_ex = name_to_onehot(['jane'], chars, max_len)
onehot_ex

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0

## Define dimension and Placeholders

In [5]:
num_data = len(data_name)
seq_len = max_len - 1
dim_data = len(chars) + 1

ph_input_name = tf.placeholder(dtype=tf.float32, shape=[None, seq_len, dim_data])
ph_output_name = tf.placeholder(dtype=tf.float32, shape=[None, seq_len, dim_data])

## Define weight variables

In [6]:
dim_rnn_cell = 128
stddev = 0.02

with tf.variable_scope('weights'):
    W_i = tf.get_variable('W_i', dtype=tf.float32,
                         initializer=tf.random_normal([dim_data, dim_rnn_cell],
                         stddev = stddev))
    b_i = tf.get_variable('b_i', dtype=tf.float32,
                         initializer=tf.random_normal([dim_rnn_cell],
                         stddev = stddev))
    W_o = tf.get_variable('W_o', dtype=tf.float32,
                         initializer=tf.random_normal([dim_rnn_cell, dim_data],
                         stddev = stddev))
    b_o = tf.get_variable('b_o', dtype=tf.float32,
                         initializer=tf.random_normal([dim_data],
                         stddev = stddev))

## Define RNN for training

In [21]:
def name_rnn_train(_x, _seq_len, _dim_data, _dim_rnn_cell):
    _x_split = tf.transpose(_x, [1, 0, 2]) # seq_len, batch, dim_data
    _x_split = tf.reshape(_x_split, [-1, _dim_data])
        
    # Linear Operation for Input
    # Fill here
       
    # Define LSTM Cell && RNN
    with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
        _rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(_dim_rnn_cell)
        _output, _state = tf.nn.static_rnn(_rnn_cell, _h_split, dtype=tf.float32)
    
    # Linear Operation for Output
    _total_out = []
    # Fill here
        
    return tf.transpose(tf.stack(_total_out), [1, 0, 2])

## Define result graph

In [17]:
result_name = name_rnn_train(ph_input_name, seq_len, dim_data, dim_rnn_cell)
print('result_shape :', result_name.shape)

result_shape : (?, 10, 27)


## Define Loss function

In [18]:
def name_loss(_gt_name, _result_name, _seq_len):
    total_loss = 0
    for i in range(_seq_len):
        # Fill here
    return total_loss
rnn_loss = name_loss(ph_output_name, result_name, seq_len)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



## Define Optimizer and Get Ready

In [19]:
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(rnn_loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=tf.trainable_variables())

print('Now ready to start the session')

Now ready to start the session


## Session Run

In [20]:
max_epoch = 300
batch_size = 64
num_batch = int(num_data/batch_size)
with tf.Session() as sess:
    sess.run(init)
    
    for _epoch in range(max_epoch):
        random.seed(_epoch)
        batch_shuffle = list(range(num_data))
        random.shuffle(batch_shuffle)
        
        total_train_loss = 0
        
        for i in range(num_batch):
            batch_idx = [batch_shuffle[idx] for idx in range(i*batch_size,
                                                            (i+1)*batch_size)]
            batch_names = [name for name in data_name if data_name.index(name) in batch_idx]
            batch_onehots = name_to_onehot(batch_names, chars, max_len)
            
            input_onehot = batch_onehots[:, 0:(max_len-1), :]
            output_onehot = batch_onehots[:, 1:max_len, :]

            train_feed_dict = {ph_input_name: input_onehot, 
                               ph_output_name: output_onehot}
            
            sess.run(optimizer, feed_dict = train_feed_dict)
            curr_loss = sess.run(rnn_loss, feed_dict=train_feed_dict)
            
            total_train_loss += curr_loss/num_batch

        print('epoch : {}, train_loss : {}'.format(_epoch+1, total_train_loss))
        
        model_save_path = saver.save(sess, './RNN_model/model.ckpt', global_step=_epoch+1)
        print('Model saved in file: {}'.format(model_save_path))

epoch : 1, train_loss : 28.54256660059879
Model saved in file: ./RNN_model/model.ckpt-1
epoch : 2, train_loss : 18.67396987111945
Model saved in file: ./RNN_model/model.ckpt-2
epoch : 3, train_loss : 17.6947021484375
Model saved in file: ./RNN_model/model.ckpt-3
epoch : 4, train_loss : 17.15771614877801
Model saved in file: ./RNN_model/model.ckpt-4
epoch : 5, train_loss : 16.822392915424548
Model saved in file: ./RNN_model/model.ckpt-5
epoch : 6, train_loss : 16.50236872622841
Model saved in file: ./RNN_model/model.ckpt-6
epoch : 7, train_loss : 16.15971851348877
Model saved in file: ./RNN_model/model.ckpt-7
epoch : 8, train_loss : 15.863567603261846
Model saved in file: ./RNN_model/model.ckpt-8
epoch : 9, train_loss : 15.546000480651854
Model saved in file: ./RNN_model/model.ckpt-9
epoch : 10, train_loss : 15.251013906378496
Model saved in file: ./RNN_model/model.ckpt-10
epoch : 11, train_loss : 14.958294015181693
Model saved in file: ./RNN_model/model.ckpt-11
epoch : 12, train_loss :

epoch : 92, train_loss : 7.5594291937978655
Model saved in file: ./RNN_model/model.ckpt-92
epoch : 93, train_loss : 7.497586049531637
Model saved in file: ./RNN_model/model.ckpt-93
epoch : 94, train_loss : 7.4298533640409765
Model saved in file: ./RNN_model/model.ckpt-94
epoch : 95, train_loss : 7.387549375232897
Model saved in file: ./RNN_model/model.ckpt-95
epoch : 96, train_loss : 7.342127197667171
Model saved in file: ./RNN_model/model.ckpt-96
epoch : 97, train_loss : 7.279497246993215
Model saved in file: ./RNN_model/model.ckpt-97
epoch : 98, train_loss : 7.227611642134817
Model saved in file: ./RNN_model/model.ckpt-98
epoch : 99, train_loss : 7.185787100540964
Model saved in file: ./RNN_model/model.ckpt-99
epoch : 100, train_loss : 7.120706583324233
Model saved in file: ./RNN_model/model.ckpt-100
epoch : 101, train_loss : 7.06613658603869
Model saved in file: ./RNN_model/model.ckpt-101
epoch : 102, train_loss : 7.016613608912419
Model saved in file: ./RNN_model/model.ckpt-102
epo

epoch : 182, train_loss : 5.035394317225406
Model saved in file: ./RNN_model/model.ckpt-182
epoch : 183, train_loss : 5.022335805391011
Model saved in file: ./RNN_model/model.ckpt-183
epoch : 184, train_loss : 5.015330239346152
Model saved in file: ./RNN_model/model.ckpt-184
epoch : 185, train_loss : 5.001781639299894
Model saved in file: ./RNN_model/model.ckpt-185
epoch : 186, train_loss : 4.991091376856755
Model saved in file: ./RNN_model/model.ckpt-186
epoch : 187, train_loss : 4.975522994995117
Model saved in file: ./RNN_model/model.ckpt-187
epoch : 188, train_loss : 4.9709730650249275
Model saved in file: ./RNN_model/model.ckpt-188
epoch : 189, train_loss : 4.958301217932451
Model saved in file: ./RNN_model/model.ckpt-189
epoch : 190, train_loss : 4.954532548000939
Model saved in file: ./RNN_model/model.ckpt-190
epoch : 191, train_loss : 4.949857485921759
Model saved in file: ./RNN_model/model.ckpt-191
epoch : 192, train_loss : 4.932167404576353
Model saved in file: ./RNN_model/mo

epoch : 272, train_loss : 4.581478319670024
Model saved in file: ./RNN_model/model.ckpt-272
epoch : 273, train_loss : 4.599064099161248
Model saved in file: ./RNN_model/model.ckpt-273
epoch : 274, train_loss : 4.6004664019534465
Model saved in file: ./RNN_model/model.ckpt-274
epoch : 275, train_loss : 4.581155300140381
Model saved in file: ./RNN_model/model.ckpt-275
epoch : 276, train_loss : 4.590859287663509
Model saved in file: ./RNN_model/model.ckpt-276
epoch : 277, train_loss : 4.590206020756772
Model saved in file: ./RNN_model/model.ckpt-277
epoch : 278, train_loss : 4.578938358708433
Model saved in file: ./RNN_model/model.ckpt-278
epoch : 279, train_loss : 4.5749553881193465
Model saved in file: ./RNN_model/model.ckpt-279
epoch : 280, train_loss : 4.570446190081145
Model saved in file: ./RNN_model/model.ckpt-280
epoch : 281, train_loss : 4.578030310179058
Model saved in file: ./RNN_model/model.ckpt-281
epoch : 282, train_loss : 4.576507693842838
Model saved in file: ./RNN_model/m

## Define RNN to test

In [23]:
ph_test_input_name = tf.placeholder(dtype=tf.float32, shape=[1, 1, dim_data])
ph_h = tf.placeholder(dtype=tf.float32, shape=[1, dim_rnn_cell])
ph_c = tf.placeholder(dtype=tf.float32, shape=[1, dim_rnn_cell])

def name_rnn_test(_x, _dim_data, _dim_rnn_cell, _prev_h, _prev_c):
    _x_split = tf.transpose(_x, [1, 0, 2]) # seq_len, batch, dim_data
    _x_split = tf.reshape(_x_split, [-1, _dim_data])
    
    # Linear Operation for Input
    # Fill here

    # Define LSTM Cell && RNN including (h, c)
    # Fill here
    
    # Linear Operation for Ouput
    _total_out = []    
    # Fill here

## Run Test

In [24]:
test_result_name, test_state = name_rnn_test(ph_test_input_name, dim_data, dim_rnn_cell,
                                            ph_h, ph_c)

with tf.Session() as sess:
    sess.run(init)
    
    saver.restore(sess, './RNN_model/model.ckpt-300')
    
    total_name = ''
    prev_char = 'a'
    total_name += prev_char
    prev_state = (np.zeros((1, dim_rnn_cell)), np.zeros((1, dim_rnn_cell)))
    for i in range(seq_len):
        input_onehot = np.zeros((1, 1, dim_data))
        prev_char_idx = chars.index(prev_char)
        input_onehot[:, :, prev_char_idx] = 1
            
        test_feed_dict = {ph_test_input_name: input_onehot,
                          ph_h: prev_state[0], ph_c: prev_state[1]}
        curr_result, curr_state = sess.run([test_result_name, test_state], test_feed_dict)
        if np.argmax(curr_result) == dim_data-1:
            break
        else:
            softmax_result = sess.run(tf.nn.softmax(test_result_name), test_feed_dict)
            softmax_result = np.squeeze(softmax_result)
            softmax_result = softmax_result[:dim_data-1]/sum(softmax_result[:dim_data-1])
            prev_char = np.random.choice(chars, 1, p=softmax_result)
            total_name += prev_char[0]
            prev_state = curr_state        
    print('Result Name :', total_name)

INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-300
Result Name : anique
